In [21]:
import pandas as pd
import copy as cp
from customfunctions import *

mapdoc=pd.read_excel('data/mapping/team_map.xlsx')
mapdic={}
for idx, row in mapdoc.iterrows():
    mapdic[row['oddsteams']]=row['nflscrapr']

oddshome=pd.read_excel('data/odds/nfl.xlsx')

oddshome['home']=oddshome['Home Team'].map(mapdic)
oddshome['away']=oddshome['Away Team'].map(mapdic)
oddshome['date'] = pd.to_datetime(oddshome['Date'], format='%Y-%m-%d')

oddsaway=cp.copy(oddshome)

oddshome['to']='home'
oddsaway['to']='away'
oddshome['outcome_dif']=oddshome['Home Score']-oddshome['Away Score']
oddsaway['outcome_dif']=oddsaway['Away Score']-oddshome['Home Score']

spoddshome=cp.copy(oddshome)
spoddsaway=cp.copy(oddsaway)

oddshome['spread']=0
oddsaway['spread']=0
spoddshome['spread']=spoddshome['Home Line Open']
spoddsaway['spread']=spoddsaway['Away Line Open']

oddshome['prob']=oddshome['Home Odds Open'].apply(dectoprob)
oddsaway['prob']=oddsaway['Away Odds Open'].apply(dectoprob)
spoddshome['prob']=spoddshome['Home Line Odds Open'].apply(dectoprob)
spoddsaway['prob']=spoddsaway['Away Line Odds Open'].apply(dectoprob)

for df in [oddshome,oddsaway,spoddshome,spoddsaway]:
    df['outcome_bin']=(df['outcome_dif']>-df['spread']).astype(int)

main=pd.concat([oddshome,oddsaway,spoddshome,spoddsaway], ignore_index=True)
main['amt']=0
main=main[main['date']>='2014-09-04'][['home','away','to','date','spread','prob','outcome_dif','outcome_bin','amt']]

main.to_csv('data/odds/hist_odds.csv')